<a href="https://colab.research.google.com/github/fx757887/100knocks-preprocess_ForColab-AzureNotebook/blob/master/cucumber_cnn_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/workpiles/CUCUMBER-9.git

Cloning into 'CUCUMBER-9'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.


In [ ]:
# ファイルを解凍する。
# prototype_1
!tar -zxvf ./CUCUMBER-9/prototype_1/cucumber-9-python.tar.gz

batches.meta
data_batch_5
data_batch_4
data_batch_3
data_batch_2
data_batch_1
test_batch


In [ ]:
!mkdir weights

In [ ]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, BatchNormalization, LeakyReLU 
from keras.layers import Reshape, Conv2DTranspose, Activation, MaxPooling2D
from keras.layers import Flatten, Dropout, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [ ]:
def load_image_and_label(pickled_files):
    # 各ファイルには495画像が含まれている
    IMAGE_COUNT_PER_FILE = 495
    # 画像の形状 : 32x32x3
    ROW = 32
    COL = 32
    DIM = 3
    whole_images = np.empty((IMAGE_COUNT_PER_FILE*len(pickled_files), ROW, COL, DIM))
    whole_labels = np.empty(IMAGE_COUNT_PER_FILE*len(pickled_files))
    for i, pickled_file in enumerate(pickled_files):
        dict = _unpickle(pickled_file)
        images = dict['data'].reshape(IMAGE_COUNT_PER_FILE, DIM, ROW, COL).transpose(0, 2, 3, 1)
        whole_images[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE, :, :, :] = images
        labels = dict['labels']
        whole_labels[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE] = labels
    return (whole_images, whole_labels)

In [ ]:
def _unpickle(pickled_file):
    import pickle

    with open(pickled_file, 'rb') as file:
        dict = pickle.load(file, encoding='latin1')
    return dict

In [ ]:
def load_cucumber():
    (X_train, y_train) = load_image_and_label(['data_batch_1',
                                     'data_batch_2',
                                     'data_batch_3',
                                     'data_batch_4',
                                     'data_batch_5',])
    (X_test, y_test) = load_image_and_label(['test_batch'])
    
    return X_train, X_test, y_train, y_test

In [ ]:
def normalize(X):
    return (X-127.5)/127.5

In [ ]:
### データの準備
X_train, X_test, y_train, y_test = load_cucumber()
X_train = normalize(X_train)
X_test = normalize(X_test)
input_shape = X_train[0].shape
X_test_original = X_test.copy()

y_train=keras.utils.to_categorical(y_train,9)
y_test=keras.utils.to_categorical(y_test,9)


In [ ]:
#画像をtrain,validation,testディレクトリに分割
X_train2,X_val,y_train2,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=0)


In [ ]:
(2475, 32, 32, 3)
(2475, 9)
(495, 32, 32, 3)
(495, 9)
(495, 32, 32, 3)
(495, 9)

(495, 9)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2475, 32, 32, 3)
(2475, 9)
(495, 32, 32, 3)
(495, 9)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=3000
nb_class=9

for i in range(100):
  train_datagen = ImageDataGenerator(
      width_shift_range=0.2,
      rotation_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      shear_range=0.1,
      fill_mode='constant',
      cval=1)
  
  train_datagen.fit(X_train)
  g = train_datagen.flow(X_train, y_train, batch_size=128, shuffle=False)
  X_batch, y_batch = g.next()
  X_train = np.append(X_train, X_batch, axis=0)
  y_train = np.append(y_train, y_batch, axis=0)

test_dationgen = ImageDataGenerator()


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(15275, 32, 32, 3)
(15275, 9)
(495, 32, 32, 3)
(495, 9)


In [ ]:
from keras import layers
from keras import models

# モデルの定義
model = models.Sequential()

model.add(layers.Conv2D(64, (2,2),activation='relu',input_shape=(32,32,3)))
model.add(layers.Conv2D(64, (3,3),padding='same',activation='relu'))
model.add(layers.Conv2D(64, (4,4),padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128,(2,2),padding='same',activation='relu'))
model.add(layers.Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(layers.Conv2D(128,(4,4),padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256,(2,2),padding='same', activation='relu'))
model.add(layers.Conv2D(256,(3,3),padding='same', activation='relu'))
model.add(layers.Conv2D(256,(4,4),padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(9, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 64)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        65600     
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 31, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 31, 31, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 64)        0

In [ ]:
#VGG16モデルの畳み込みベースのインスタンス化
from keras.applications import VGG16
conv_base=VGG16(weights='imagenet',
                include_top=False,
                input_shape=(150,150,3))

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
#5-20
from keras import models
from keras import layers

model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print('This is the number of trainable weights before freezing the conv base:',len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [ ]:
conv_base.trainable=False

In [ ]:
print('This is the number of trainable weights after freezing the conv base:',len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

In [ ]:
# NNの反復回数、バッチのサイズを決定
epochs = 200
batch_size = 128

# 学習を実行します。結果はhistoryに保存されます
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=test_dationgen
                    
                    .flow(X_test,y_test),
                    validation_steps=len(X_test)/128,
                    shuffle=True,
                    )

# save model

model.save('model')

Epoch 1/200


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))
plt.plot(epochs, acc, 'ro', label='T acc')
plt.plot(epochs, val_acc, 'g', label='V acc')
plt.legend();

In [ ]:
epochs=range(len(acc))
plt.plot(epochs, loss, 'ro', label='T loss')
plt.plot(epochs, val_loss, 'g', label='V loss')
plt.legend();

